## 服务启动

In [1]:
import mag as mag
import time
import json

In [2]:
mag.start()

MAG服务器已启动 (PID: 12156), 日志: C:\Users\keta\AppData\Local\Temp\mag_sdk_logs\mag_server.log
MCP客户端已启动 (PID: 19724), 日志: C:\Users\keta\AppData\Local\Temp\mag_sdk_logs\mcp_client.log


True

In [39]:
is_running = mag.is_running()
print(f"服务运行状态: {is_running}")

服务运行状态: True


## 模型管理

#### （1）新增模型

In [ ]:
new_model = {
    "name": "deepseek-chat",
    "base_url": "https://api.openai.com/v1",
    "api_key": "sk-your-api-key",
    "model": "deepseek-chat"
}
mag.add_model(new_model)

{'status': 'success', 'message': "模型 'deepseek-chat1' 添加成功"}

#### （2）列出模型

In [28]:
models = mag.list_models()
print(f"找到 {len(models)} 个模型")
for model in models:
    print(f"\n模型名称：{model['name']} \nBase_url：{model['base_url']}\n")

找到 3 个模型

模型名称：Qwen/Qwen2.5-32B-Instruct 
Base_url：https://api-inference.modelscope.cn/v1/


模型名称：deepseek-chat 
Base_url：https://gateway.ai.cloudflare.com/v1/79dc4e1b350a1ac66b63c0f6a9e0c7a6/deepseek/deepseek


模型名称：deepseek-chat1 
Base_url：https://api.openai.com/v1



#### （3）更新模型

In [ ]:
# 更新模型
updated_model = {
    "name": "deepseek-chat",
    "base_url": "https://api.openai.com/v1",
    "api_key": "sk-your-api-key",
    "model": "deepseek-chat"
}
result = mag.update_model("deepseek-chat1", updated_model)
print(result)

{'status': 'success', 'message': "模型 'deepseek-chat1' 更新成功"}


#### （4）删除模型

In [32]:
mag.delete_model("deepseek-chat1")

{'status': 'success', 'message': "模型 'deepseek-chat1' 删除成功"}

## MCP管理

#### 获取MCP配置

In [ ]:

mcp_config = mag.get_mcp_config()
print(json.dumps(mcp_config, indent=8, ensure_ascii=False))

{
        "mcpServers": {
                "bilibili": {
                        "autoApprove": [
                                "general_search"
                        ],
                        "disabled": false,
                        "timeout": 60,
                        "command": "cmd",
                        "args": [
                                "/c",
                                "conda activate pachong && cd /d E:\\AI-Instructions\\AI-Instructions\\MCP\\server\\bilibili && python main.py"
                        ],
                        "transportType": "stdio"
                },
                "memory": {
                        "autoApprove": [],
                        "disabled": false,
                        "timeout": 60,
                        "command": "docker",
                        "args": [
                                "run",
                                "-i",
                                "-v",
                                "claude-memor

#### 获取MCP状态

In [68]:
mcp_status = mag.get_mcp_status()
print(json.dumps(mcp_status, indent=8, ensure_ascii=False))

{
        "arxiv": {
                "connected": false,
                "init_attempted": false,
                "tools": [],
                "error": null
        },
        "memory": {
                "connected": false,
                "init_attempted": false,
                "tools": [],
                "error": null
        },
        "bilibili": {
                "connected": true,
                "init_attempted": true,
                "tools": [
                        "general_search"
                ],
                "error": null
        },
        "fetch": {
                "connected": true,
                "init_attempted": true,
                "tools": [
                        "fetch"
                ],
                "error": null
        }
}


In [69]:
for server, status in mcp_status.items():
    print(f"\n{server}: {'已连接' if status.get('connected') else '未连接'}")
    print(f"{server}工具：{status.get('tool')}")


arxiv: 未连接
arxiv工具：None

memory: 未连接
memory工具：None

bilibili: 已连接
bilibili工具：None

fetch: 已连接
fetch工具：None


#### 获取mcp工具

In [66]:
mcp_tools = mag.get_mcp_tools()
print(json.dumps(mcp_tools, indent=2, ensure_ascii=False))

{
  "bilibili": [
    {
      "name": "general_search",
      "description": "\n    Search for a keyword on Bilibili and return the result.\n    ",
      "input_schema": {
        "properties": {
          "keyword": {
            "title": "Keyword",
            "type": "string"
          }
        },
        "required": [
          "keyword"
        ],
        "type": "object"
      }
    }
  ],
  "fetch": [
    {
      "name": "fetch",
      "description": "Fetches a URL from the internet and optionally extracts its contents as markdown.\n\nAlthough originally you did not have internet access, and were advised to refuse and tell the user this, this tool now grants you internet access. Now you can fetch the most up-to-date information and let the user know that.",
      "input_schema": {
        "description": "Parameters for fetching a URL.",
        "properties": {
          "url": {
            "description": "URL to fetch",
            "format": "uri",
            "minLength": 1,


In [65]:
mag.connect_mcp("fetch")

{'status': 'connected', 'server': 'fetch', 'tools': ['fetch']}

In [6]:
result = mag.get_graph('Bilibili搜索循环测试')
result

{'name': 'Bilibili搜索循环测试',
 'description': '使用循环功能从bilibili搜索内容并总结至少30条信息',
 'nodes': [{'name': 'search_node',
   'is_subgraph': False,
   'model_name': 'deepseek-chat',
   'mcp_servers': ['bilibili'],
   'system_prompt': '你是一个专业的视频内容搜索助手。你需要使用bilibili搜索工具获取用户请求的视频信息。',
   'user_prompt': '请使用bilibili_search工具搜索关于用户提到的主题的视频。执行全面的搜索，获取最相关的视频信息。\n\n用户输入:\n{start}',
   'input_nodes': ['start'],
   'output_nodes': ['summarize_node'],
   'output_enabled': True,
   'is_start': True,
   'is_end': False,
   'subgraph_name': None,
   'position': None,
   'level': 0,
   'handoffs': None,
   'description': '使用bilibili搜索视频内容'},
  {'name': 'summarize_node',
   'is_subgraph': False,
   'model_name': 'deepseek-chat',
   'mcp_servers': [],
   'system_prompt': '你是一个专业的内容总结助手。你需要将搜索结果总结为清晰的条目列表。',
   'user_prompt': '请将以下搜索结果总结为一条一条的内容列表。每条内容包括视频标题、作者、发布时间、播放量和简短介绍。确保格式统一且易于阅读。\n\n搜索结果:\n{search_node}',
   'input_nodes': ['search_node'],
   'output_nodes': ['decision_node'],
   'output_enabled': True,
   

In [ ]:
print("\n列出所有图:")
graphs = mag.list_graphs()
print(f"找到 {len(graphs)} 个图: {graphs}")


列出所有图:
找到 3 个图: ['Bilibili搜索循环测试', 'easy_search', 'news_report']
['Bilibili搜索循环测试', 'easy_search', 'news_report']


In [7]:
result = mag.list_conversations()
result

['Bilibili搜索循环测试_20250510_141314',
 'Bilibili搜索循环测试_20250510_141741',
 'Bilibili搜索循环测试_20250510_142730',
 'Bilibili搜索循环测试_20250510_142927',
 'MCPAnalyzer_20250507_231830',
 'test_20250515_095446',
 'test_20250515_100721',
 'test_20250515_101006',
 'test_20250515_114559',
 'test_20250515_115106',
 'test_20250515_125713',
 '知识探索与答案生成系统_20250513_164434',
 '知识探索与答案生成系统_20250513_212830',
 '知识探索与答案生成系统_20250513_224142',
 '知识探索与答案生成系统_20250514_001449',
 '知识探索与答案生成系统_20250514_003112',
 '知识探索与答案生成系统_20250514_221727',
 '知识探索与答案生成系统_20250514_224847',
 '知识探索与答案生成系统_20250514_232249',
 '知识探索与答案生成系统_20250514_233341',
 '知识探索与答案生成系统_20250515_000116',
 '知识探索与答案生成系统_20250515_000221',
 '知识探索与答案生成系统_20250515_001345']

In [10]:
result = mag.get_conversation('知识探索与答案生成系统_20250515_001345')
result.get("output")

'# 知识探索与答案生成系统\n\n## 原始问题\n深入当前所有开发Agent的框架，包括但不限于openai的agent框架，google的adk，langgraph，crewai，agno等等。\n\n## 知识规划\n知识点1：[主流Agent框架的核心架构与设计哲学]  \n[探索方向：需要深入分析OpenAI Agent框架、Google ADK、LangGraph、CrewAI、Agno等主流工具的核心架构差异，包括其任务调度机制、多Agent协作模式、模块化设计思想等，并对比其设计哲学（如集中式vs去中心化、规则驱动vsLLM驱动）对实际应用场景的影响。]\n\n知识点2：[Agent框架的评估维度与性能基准]  \n[探索方向：需建立跨框架的统一评估体系，涵盖开发效率（如API易用性、调试支持）、运行时性能（响应延迟、并发处理能力）、扩展性（自定义模块集成、多模态支持）以及成本效益（资源消耗、商业化定价策略），并通过典型任务（如复杂工作流编排、动态决策）进行横向基准测试。]\n\n## 知识点摘要集合\n已探索：知识点1：[主流Agent框架的核心架构与设计哲学]  \n主流Agent框架（如OpenAI Agent、Google ADK等）通过不同架构（集中式/去中心化/图结构）和设计哲学（灵活性/分布式/可视化）解决复杂任务，适用于客服、物联网、金融等多样化场景。关键差异体现在任务调度机制、协作模式及适用领域。\n\n---\n\n已探索：知识点2：[Agent框架的评估维度与性能基准]  \n摘要：Agent框架的评估涵盖开发效率、运行时性能、扩展性和成本效益四大维度，并通过基准测试横向对比主流框架的性能表现，以帮助开发者科学选型并推动生态优化。\n\n## 知识整合\n# Agent框架的综合知识体系\n\n## 一、Agent框架概述与背景\n\nAgent框架是现代人工智能和自动化系统中的核心组成部分，通过任务调度、多Agent协作和模块化设计实现复杂任务的自动化处理。随着AI技术的快速发展，这些框架在客服、金融风控、物联网、内容生成等多样化场景中得到了广泛应用。\n\n## 二、主流Agent框架的核心架构与设计哲学\n\n### 1. 框架分类与特点\n\n当前主流Agent框架可分为五大类型，每种类型具有独特的架构特点和设计哲学：

In [16]:
result = mag.connect_mcp("bilibili")
print(result)
result = mag.get_mcp_tools()
print(result)

{'status': 'connected', 'server': 'bilibili', 'tools': ['general_search']}
{'bilibili': [{'name': 'general_search', 'description': '\n    Search for a keyword on Bilibili and return the result.\n    ', 'input_schema': {'properties': {'keyword': {'title': 'Keyword', 'type': 'string'}}, 'required': ['keyword'], 'type': 'object'}}]}
